In [ ]:
import re
import requests
import pandas as pd
from hashlib import md5
import os
from threading import Thread, Lock
from time import sleep
from datetime import datetime

In [ ]:
def get_proxy_gen(file='proxy.txt'):
    while True:
        with open(file, 'r') as f:
            for proxy in f:
                yield proxy.strip()

def get_proxy():
    global proxer
    global proxer_lock
    proxer_lock.acquire()
    proxy = next(proxer)
    proxer_lock.release()
    return proxy

def get_html_page(url):
    while True:
        try:
            proxies = {
                'https': 'http://' + get_proxy(),
            }
            req = requests.get(url, proxies=proxies)
            if (not req.ok) or (req.text.find('Доступ временно заблокирован') > 0) or\
                    (req.text.find('Доступ с вашего IP-адреса временно ограничен') > 0):
                continue
            return req
        except Exception as e:
#             print(e)
            continue
proxer = get_proxy_gen()
proxer_lock = Lock()

In [ ]:
def urls_empty():
    global urls
    global urls_lock
    
    urls_lock.acquire()
    
    ans = False
    if len(urls) == 0:
        ans = True
        
    urls_lock.release()
    
    return ans
    
def save_data():
    global data
    global data_lock
    
    data_lock.acquire()
    
    df = pd.DataFrame(data)
    df.to_csv(os.path.join('banned', 'data{}.csv'.format(str(datetime.now()).replace(':', ''))))
    data = {'url':[], 'banned':[], 'comment':[]}
    
    data_lock.release()

    
def add_data(url, banned, comment):
    global data
    global data_lock
    global num_acks
    
    data_lock.acquire()
    num_acks += 1
    if num_acks % 100 == 0:
        print(num_acks)
    data['url'].append(url)
    data['banned'].append(banned)
    data['comment'].append(comment)
    
    data_lock.release()
    
    
def get_url():
    global urls
    global urls_lock
    urls_lock.acquire()
    if len(urls) == 0:
        urls_lock.release()
        return None
    url = urls.pop()
    urls_lock.release()
    return url

    
def get_info():
    global HASH_MOD
    global alerts
    
    while True:
        url = get_url()
        if url is None:
            return
        if int(md5(url.encode('utf-8')).hexdigest(), base=16) % 3 != HASH_MOD:
            continue
        
        page = get_html_page(url)
        for al in alerts:
            if page.text.find(al) > 0:
                add_data(url, "1", al)
                break
        else:
            add_data(url, "0", "")


In [ ]:
urls = list(pd.read_csv("data_06_12.csv")['url'])
urls_lock = Lock()
data = {'url':[], 'banned':[], 'comment':[]}
data_lock = Lock()
alerts = ['Ой! Такой страницы на нашем сайте нет :(', 'Страница не найдена', 'Пользователь заблокирован']
num_acks = 0

In [ ]:
############################################################################
HASH_MOD = 0
NUM_THREADS = 50
############################################################################

for i in range(NUM_THREADS):
    t = Thread(target=get_info)
    t.start()
while True:
    sleep(60)
    if urls_empty():
        sleep(60)
        save_data()
        break
    else:
        save_data()